This outputs the cdf of a point in W for a given model.

In [11]:
import numpy as np
np.set_printoptions(suppress=True, linewidth=200)
import os
from scipy import interpolate
import warnings
warnings.filterwarnings("ignore")
import argparse
import pandas as pd


In [33]:
def return_solution(shock_expo, seed, chiUnderline, a_e, a_h, gamma_e, gamma_h, psi_e, psi_h, delta_e, delta_h, lambda_d, nu,n_layers,units,iter_num, points_size,penalization, action_name):
        
        wMin = 0.01
        wMax = 0.99

        nWealth           = 180
        nZ                = 30
        nV                = 30
        
        wMin_t, wMax_t = [str("{:0.3f}".format(param)).replace('.', '', 1)  for param in [wMin, wMax]]
        domain_folder = 'nW_' + str(nWealth) + '_nZ_' + str(nZ) + '_nV_' + str(nV) + '_wMin_' + wMin_t + '_wMax_' + wMax_t

        parameter_list    = [chiUnderline, a_e, a_h, gamma_e, gamma_h, psi_e, psi_h, delta_e, delta_h, lambda_d, nu]
        chiUnderline, a_e, a_h, gamma_e, gamma_h, psi_e, psi_h, delta_e, delta_h, lambda_d, nu = [str("{:0.3f}".format(param)).replace('.', '', 1)  for param in parameter_list]
        model_folder = 'chiUnderline_' + chiUnderline + '_a_e_' + a_e + '_a_h_' + a_h  + '_gamma_e_' + gamma_e + '_gamma_h_' + gamma_h + '_rho_e_' + psi_e + '_rho_h_' + psi_h + '_delta_e_' + delta_e + '_delta_h_' + delta_h + '_lambda_d_' + lambda_d + '_nu_' + nu
        layer_folder =  'seed_' + str(seed) + '_n_layers_' + str(n_layers) + '_units_' + str(units) +'_points_size_' + str(points_size) + '_iter_num_' + str(iter_num) + '_penalization_' + str(penalization)

        workdir = os.getcwd()
        outputdir = workdir + '/output/' + action_name + '/' + shock_expo + '/'+ domain_folder + '/' + model_folder + '/' + layer_folder + '/'
        print(outputdir)

        eva_V_10 = np.load(outputdir + 'eva_V_10.npz')
        eva_V_50 = np.load(outputdir + 'eva_V_50.npz')
        eva_V_90 = np.load(outputdir + 'eva_V_90.npz')

        print('HJBE_validation_MSE: ', np.load(outputdir + 'HJBE_validation_MSE.npy'))
        print('HJBH_validation_MSE: ', np.load(outputdir + 'HJBH_validation_MSE.npy'))
        print('kappa_validation_MSE: ', np.load(outputdir + 'kappa_validation_MSE.npy'))

        try:
                elasticity_logw = np.load(outputdir + 'elasticity_logw.npz', allow_pickle=True)
        except:
            elasticity_logw = None
        try:
            uncertaintye_priceelas = np.load(outputdir + 'uncertaintye_priceelas.npz', allow_pickle=True)
        except:
                uncertaintye_priceelas = None
        try:
            uncertaintyh_priceelas = np.load(outputdir + 'uncertaintyh_priceelas.npz', allow_pickle=True)
        except:
                uncertaintyh_priceelas = None

        W = np.load(outputdir + 'W_NN.npy')
        W = pd.DataFrame(W, columns = ['W'])
        Z = np.load(outputdir + 'Z_NN.npy')
        Z = pd.DataFrame(Z, columns = ['Z'])
        V = np.load(outputdir + 'V_NN.npy')
        V = pd.DataFrame(V, columns = ['V'])

        dent = np.load(outputdir + 'dent_NN.npy')
        dent_org = dent
        dent = pd.DataFrame(dent, columns = ['dent'])
        dent = pd.concat([W,Z,V,dent['dent']], axis=1)
        dentW = dent.groupby('W').sum()['dent']
        dentV = dent.groupby('V').sum()['dent']

        try:
                elasticities_W_percentile_005 = np.load(outputdir + 'elasticities_W_percentile_0.05.npz', allow_pickle=True)
        except:
                elasticities_W_percentile_005 = None
        try:
                elasticities_W_percentile_01 = np.load(outputdir + 'elasticities_W_percentile_0.1.npz', allow_pickle=True)
        except:
                elasticities_W_percentile_01 = None
        try:
                elasticities_W_percentile_05 = np.load(outputdir + 'elasticities_W_percentile_0.5.npz', allow_pickle=True)
        except:
                elasticities_W_percentile_05 = None

        return {'eva_V_10':eva_V_10, 'eva_V_50':eva_V_50, 'eva_V_90':eva_V_90, 'dent':dent_org,'dentW':dentW, 'dentV':dentV,
                'W':W, 'Z':Z, 'V':V, 'elasticity_logw':elasticity_logw,  'uncertaintye_priceelas':uncertaintye_priceelas, 'uncertaintyh_priceelas':uncertaintyh_priceelas,\
                'elasticities_W_percentile_005':elasticities_W_percentile_005, 'elasticities_W_percentile_01':elasticities_W_percentile_01, 'elasticities_W_percentile_05':elasticities_W_percentile_05}


In [34]:
modelRF_lower = return_solution(shock_expo = 'lower_triangular', seed = 256, chiUnderline = 1.0, a_e = 0.0922, a_h = 0.0, gamma_e = 4.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')
modelRF_upper = return_solution(shock_expo = 'upper_triangular', seed = 256, chiUnderline = 1.0, a_e = 0.0922, a_h = 0.0, gamma_e = 4.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')
modelSG_lower = return_solution(shock_expo = 'lower_triangular', seed = 256, chiUnderline = 0.2, a_e = 0.0922, a_h = 0.0, gamma_e = 4.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')
modelSG_upper = return_solution(shock_expo = 'upper_triangular', seed = 256, chiUnderline = 1.0, a_e = 0.0922, a_h = 0.0, gamma_e = 4.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')
modelPR_lower = return_solution(shock_expo = 'lower_triangular', seed = 256, chiUnderline = 0.00001, a_e = 0.0922, a_h = 0.0, gamma_e = 4.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')
modelPR_upper = return_solution(shock_expo = 'upper_triangular', seed = 256, chiUnderline = 0.00001, a_e = 0.0922, a_h = 0.0, gamma_e = 4.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')

modelRF_lower_gammae_3 = return_solution(shock_expo = 'lower_triangular',seed = 256, chiUnderline = 1.0, a_e = 0.0922, a_h = 0.0, gamma_e = 3.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')
modelRF_lower_gammae_5 = return_solution(shock_expo = 'lower_triangular',seed = 256, chiUnderline = 1.0, a_e = 0.0922, a_h = 0.0, gamma_e = 5.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')

modelSG_lower_gammae_3 = return_solution(shock_expo = 'lower_triangular',seed = 256, chiUnderline = 0.2, a_e = 0.0922, a_h = 0.0, gamma_e = 3.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')
modelSG_lower_gammae_5 = return_solution(shock_expo = 'lower_triangular',seed = 256, chiUnderline = 0.2, a_e = 0.0922, a_h = 0.0, gamma_e = 5.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')

modelPR_lower_gammae_3 = return_solution(shock_expo = 'lower_triangular',seed = 256, chiUnderline = 0.00001, a_e = 0.0922, a_h = 0.0, gamma_e = 3.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')
modelPR_lower_gammae_5 = return_solution(shock_expo = 'lower_triangular',seed = 256, chiUnderline = 0.00001, a_e = 0.0922, a_h = 0.0, gamma_e = 5.0, gamma_h = 8.0, psi_e = 1.0, psi_h = 1.0, delta_e = 0.0115, delta_h = 0.01, lambda_d = 0.0, nu = 0.1, n_layers = 2, units = 16, iter_num = 5, points_size = 10, penalization = 10000, action_name = 'neural_net')


/project/lhansen/macrofinance/heterogenous_agents_with_frictions_NN/output/neural_net/lower_triangular/nW_180_nZ_30_nV_30_wMin_0010_wMax_0990/chiUnderline_1000_a_e_0092_a_h_0000_gamma_e_4000_gamma_h_8000_rho_e_1000_rho_h_1000_delta_e_0011_delta_h_0010_lambda_d_0000_nu_0100/seed_256_n_layers_2_units_16_points_size_10_iter_num_5_penalization_10000/
HJBE_validation_MSE:  2.8413337653444003e-05
HJBH_validation_MSE:  2.0985705160093152e-05
kappa_validation_MSE:  8.453717859970984e-13
/project/lhansen/macrofinance/heterogenous_agents_with_frictions_NN/output/neural_net/upper_triangular/nW_180_nZ_30_nV_30_wMin_0010_wMax_0990/chiUnderline_1000_a_e_0092_a_h_0000_gamma_e_4000_gamma_h_8000_rho_e_1000_rho_h_1000_delta_e_0011_delta_h_0010_lambda_d_0000_nu_0100/seed_256_n_layers_2_units_16_points_size_10_iter_num_5_penalization_10000/
HJBE_validation_MSE:  2.442589871103974e-05
HJBH_validation_MSE:  2.0688856497623123e-05
kappa_validation_MSE:  8.935164172208854e-13
/project/lhansen/macrofinance/het

In [130]:
def read_npy_drift_term(filename, statespace_shape, outputdir):
    """
    Load drift term from .npy file, return reshaped numpy array with the same shape as the state space grid.

    Parameters:
    filename: str
        The name of the .npy file to be loaded.
    statespace_shape: list of int
        The shape of the state space grid.
    
    Returns:
    numpy array
        The drift term as a reshaped numpy array with the same shape as the state space grid.
    """
    data = np.load(outputdir + filename + '.npy')
    return data.reshape(statespace_shape, order='F')

def read_npy_state(filename, outputdir):
    """
    Load state variable from .npy file, return unique grid points as flattened numpy array.

    Parameters:
    filename: str
        The name of the .npy file to be loaded.

    Returns:
    numpy array
        Unique grid points as flattened numpy array.
    """
    data = np.load(outputdir + filename + '.npy')
    return np.unique(data)
    
def marginal_quantile_func_factory(dent, statespace, statename):
    """
    Load stationary density from .npy file, return the marginal quantile function for each state variable.

    Parameters:
    dent: numpy array
        The stationary density as a numpy array, with the shape of the state space grid.
    statespace: list of numpy arrays
        List of state variables in numpy arrays for each state dimension. Grid points should be unique and sorted in ascending order.
    statename: list of str
        List of state variable names.
    
    Returns:
    dict
        The marginal quantile function for each state variable.
    """
    inverseCDFs = {}
    
    nRange   = list(range(len(statespace)))
    for i, state in enumerate(statespace):
        axes     = list(filter(lambda x: x != i,nRange))
        condDent = dent.sum(axis = tuple(axes))
        cumden   = np.cumsum(condDent.copy())
        cdf      = interpolate.interp1d(cumden, state, fill_value= (state.min(), state.max()), bounds_error = True)
        inverseCDFs[statename[i]] = cdf
        
    return inverseCDFs

In [124]:
state = statespace[0]

In [154]:
state[]

array([0.01      , 0.01547486, 0.02094972, 0.02642458, 0.03189944, 0.0373743 , 0.04284916, 0.04832402, 0.05379888, 0.05927374, 0.0647486 , 0.07022346, 0.07569832, 0.08117318, 0.08664804, 0.09212291,
       0.09759777, 0.10307263, 0.10854749, 0.11402235, 0.11949721, 0.12497207, 0.13044693, 0.13592179, 0.14139665, 0.14687151, 0.15234637, 0.15782123, 0.16329609, 0.16877095, 0.17424581, 0.17972067,
       0.18519553, 0.19067039, 0.19614525, 0.20162011, 0.20709497, 0.21256983, 0.21804469, 0.22351955, 0.22899441, 0.23446927, 0.23994413, 0.24541899, 0.25089385, 0.25636872, 0.26184358, 0.26731844,
       0.2727933 , 0.27826816, 0.28374302, 0.28921788, 0.29469274, 0.3001676 , 0.30564246, 0.31111732, 0.31659218, 0.32206704, 0.3275419 , 0.33301676, 0.33849162, 0.34396648, 0.34944134, 0.3549162 ,
       0.36039106, 0.36586592, 0.37134078, 0.37681564, 0.3822905 , 0.38776536, 0.39324022, 0.39871508, 0.40418994, 0.4096648 , 0.41513966, 0.42061453, 0.42608939, 0.43156425, 0.43703911, 0.44251397,
     

In [153]:
cumden[]

array([0.        , 0.        , 0.00000001, 0.00000003, 0.00000005, 0.00000009, 0.00000015, 0.00000023, 0.00000035, 0.00000052, 0.00000074, 0.00000105, 0.00000147, 0.00000202, 0.00000276, 0.00000373,
       0.000005  , 0.00000667, 0.00000883, 0.00001163, 0.00001523, 0.00001985, 0.00002573, 0.00003319, 0.00004263, 0.00005452, 0.00006942, 0.00008803, 0.00011119, 0.00013988, 0.0001753 , 0.00021889,
       0.00027231, 0.00033757, 0.00041701, 0.00051339, 0.00062992, 0.00077034, 0.00093899, 0.0011409 , 0.00138184, 0.00166845, 0.0020081 , 0.00240935, 0.00288183, 0.00343643, 0.00408535, 0.00484223,
       0.00572224, 0.0067422 , 0.00792065, 0.00927797, 0.01083646, 0.01262045, 0.01465499, 0.01696799, 0.01958934, 0.02255077, 0.02588585, 0.02962984, 0.03381961, 0.03849337, 0.04369056, 0.04945155,
       0.05581529, 0.0628204 , 0.07050682, 0.07891354, 0.08807816, 0.09803632, 0.10882121, 0.12046293, 0.13298798, 0.14641786, 0.16076283, 0.17603372, 0.19223504, 0.20936454, 0.22741275, 0.24636269,
     

In [152]:
cdf(0.01)

array(0.29175432)

In [142]:
cumden

array([0.        , 0.        , 0.00000001, 0.00000003, 0.00000005, 0.00000009, 0.00000015, 0.00000023, 0.00000035, 0.00000052, 0.00000074, 0.00000105, 0.00000147, 0.00000202, 0.00000276, 0.00000373,
       0.000005  , 0.00000667, 0.00000883, 0.00001163, 0.00001523, 0.00001985, 0.00002573, 0.00003319, 0.00004263, 0.00005452, 0.00006942, 0.00008803, 0.00011119, 0.00013988, 0.0001753 , 0.00021889,
       0.00027231, 0.00033757, 0.00041701, 0.00051339, 0.00062992, 0.00077034, 0.00093899, 0.0011409 , 0.00138184, 0.00166845, 0.0020081 , 0.00240935, 0.00288183, 0.00343643, 0.00408535, 0.00484223,
       0.00572224, 0.0067422 , 0.00792065, 0.00927797, 0.01083646, 0.01262045, 0.01465499, 0.01696799, 0.01958934, 0.02255077, 0.02588585, 0.02962984, 0.03381961, 0.03849337, 0.04369056, 0.04945155,
       0.05581529, 0.0628204 , 0.07050682, 0.07891354, 0.08807816, 0.09803632, 0.10882121, 0.12046293, 0.13298798, 0.14641786, 0.16076283, 0.17603372, 0.19223504, 0.20936454, 0.22741275, 0.24636269,
     

In [131]:
dent = modelSG_lower['dent'].reshape(180,30,30, order='F')
W = np.array(modelSG_lower['W']).reshape(180,30,30, order='F')[:,0,0]
Z = np.array(modelSG_lower['Z']).reshape(180,30,30, order='F')[0,:,0]
V = np.array(modelSG_lower['V']).reshape(180,30,30, order='F')[0,0,:]
# [W,Z,V] = [modelSG_lower[key] for key in ['W','Z','V']]
# statespace = [W['W'], Z['Z'], V['V']]
statespace = [W,Z,V]
statename = ['W','Z','V']
inverse_SG = marginal_quantile_func_factory(dent, statespace, statename)

In [134]:
inverse_SG['W'](0.012)

array(0.0636328)

array([[[ 0.,  0.,  0., ...,  0.,  0., -0.],
        [ 0.,  0.,  0., ...,  0.,  0., -0.],
        [ 0.,  0.,  0., ...,  0.,  0., -0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0., -0.],
        [ 0.,  0.,  0., ...,  0.,  0., -0.],
        [-0., -0., -0., ..., -0., -0., -0.]],

       [[ 0.,  0.,  0., ...,  0.,  0., -0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [-0.,  0.,  0., ...,  0.,  0., -0.]],

       [[-0.,  0.,  0., ...,  0.,  0., -0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [-0.,  0.,  0., ...,  0.,  0., -0.]],

       ...,

       [[-0.,  0.,  0., ...,  0.,  0., -0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.

In [137]:
dent = modelRF_lower['dent'].reshape(180,30,30, order='F')
W = np.array(modelRF_lower['W']).reshape(180,30,30, order='F')[:,0,0]
Z = np.array(modelRF_lower['Z']).reshape(180,30,30, order='F')[0,:,0]
V = np.array(modelRF_lower['V']).reshape(180,30,30, order='F')[0,0,:]
# [W,Z,V] = [modelSG_lower[key] for key in ['W','Z','V']]
# statespace = [W['W'], Z['Z'], V['V']]
statespace = [W,Z,V]
statename = ['W','Z','V']
inverse_RF = marginal_quantile_func_factory(dent, statespace, statename)

In [136]:
inverse_RF['W'](0.01)

array(0.29175432)